In [ ]:
pip install cozepy

In [ ]:
import pandas as pd

balanced_small_test = pd.read_csv("/content/balanced_small_test.csv")
balanced_df = pd.read_csv("/content/balanced_summary_dataset.csv")

## Prompts

In [ ]:
eng_prompt = """
    A hallucination occurs if the ANSWER includes information not grounded in the CONTEXT or does not directly and accurately address the QUESTION.

    Definition:
    - The ANSWER is hallucinated if it contradicts the CONTEXT, adds unsupported claims, introduces fabricated, untrue details, or fails to correctly address the QUESTION.
    - The ANSWER is faithful if it strictly adheres to the facts in the CONTEXT and:
        1. Accurately responds to the QUESTION using the provided CONTEXT, OR
        2. Clearly states that it cannot answer the QUESTION because the CONTEXT does not contain relevant information.

    Instructions:
    - Respond ONLY "true" if the ANSWER contains hallucinations.
    - Respond ONLY "false" if the ANSWER is grounded, accurate, and appropriately addresses the QUESTION.

    CONTEXT:

    =====
    {context}
    =====

    QUESTION:

    =====
    {query}
    =====

    ANSWER:

    =====
    {output}
    =====
"""

In [ ]:
eng_prompt_with_example = """
You are a fact-checking assistant. Your job is to identify whether the ANSWER contains any hallucinations based on the CONTEXT and QUESTION.

Definition:
- A hallucination occurs if the ANSWER:
  - Includes claims or facts that are not in the CONTEXT.
  - Contradicts information in the CONTEXT.
  - Fails to directly and accurately answer the QUESTION.
- An ANSWER is faithful if:
  - It answers the QUESTION based only on the CONTEXT, OR
  - It states that the CONTEXT does not contain enough information.

Instructions:
Respond only with:
- "true" if the ANSWER contains any hallucination.
- "false" if the ANSWER is completely grounded in the CONTEXT.

Example 1:
CONTEXT: Marie Curie won two Nobel Prizes.
QUESTION: How many Nobel Prizes did Marie Curie win?
ANSWER: Marie Curie won two Nobel Prizes.
→ false

Example 2:
CONTEXT: Marie Curie won two Nobel Prizes.
QUESTION: What kind of music did Marie Curie enjoy?
ANSWER: Marie Curie enjoyed classical music.
→ true

Example 3:
CONTEXT: The Great Fire of London started in a bakery on Pudding Lane in 1666. It destroyed much of the city, including 87 churches and thousands of homes. There were very few recorded deaths.
QUESTION: Summarize the event in one or two sentences.
ANSWER: The Great Fire of London in 1666 began in a bakery on Pudding Lane and caused widespread destruction, including the loss of many buildings and lives.
→ true

Example 4:
CONTEXT: The Great Fire of London started in a bakery on Pudding Lane in 1666. It destroyed much of the city, including 87 churches and thousands of homes. There were very few recorded deaths.
QUESTION: Summarize the event in one or two sentences.
ANSWER: The Great Fire of London in 1666 began in a bakery on Pudding Lane and destroyed a large part of the city, including many homes and churches. Few deaths were recorded.
→ false

Now evaluate the following:

CONTEXT:
=====
{context}
=====

QUESTION:
=====
{query}
=====

ANSWER:
=====
{output}
=====
"""


In [ ]:
rus_prompt = """
Галлюцинация возникает, если ОТВЕТ содержит информацию, не основанную на КОНТЕКСТЕ, или если он не напрямую и не точно отвечает на ВОПРОС.

Определение:
- ОТВЕТ считается галлюцинированным, если он противоречит КОНТЕКСТУ, добавляет неподтвержденные утверждения, включает вымышленные или ложные детали, или не отвечает правильно на ВОПРОС.
- ОТВЕТ считается достоверным, если он строго придерживается фактов из КОНТЕКСТА и:
  1. Точно отвечает на ВОПРОС, используя предоставленный КОНТЕКСТ, ИЛИ
  2. Ясно указывает, что не может ответить на ВОПРОС, потому что в КОНТЕКСТЕ отсутствует соответствующая информация.

Инструкции:
- Ответьте ТОЛЬКО "true", если ОТВЕТ содержит галлюцинации.
- Ответьте ТОЛЬКО "false", если ОТВЕТ основан на фактах, точен и правильно отвечает на ВОПРОС.

КОНТЕКСТ:

=====
{context}
=====

ВОПРОС:

=====
{query}
=====

ОТВЕТ:

=====
{output}
=====
"""

In [ ]:
summary_prompt = """
You are an expert fact-checker and summarization analyst. Your task is to determine whether the SUMMARY accurately reflects the information in the CONTEXT. You are precise, strict, and err on the side of caution when evaluating unsupported or implied information.

A hallucination occurs if the SUMMARY includes information not grounded in the CONTEXT.

Definition:
- The SUMMARY is hallucinated if it:
  - Contradicts the CONTEXT,
  - Adds details or claims that are not supported by the CONTEXT,
  - Infers or implies facts not clearly stated in the CONTEXT,
  - Exaggerates or distorts facts from the CONTEXT.
- The SUMMARY is faithful if it:
  - Accurately reflects the facts from the CONTEXT,
  - Avoids adding or inferring unsupported information.
  - Stays true to the **spirit** and **details** of the CONTEXT, without changing the meaning, tone, or significance of the information.
  - Does not include any new details, facts, or conclusions not directly supported by the CONTEXT.

Instructions:
- Respond ONLY with:
  - "hallucinated" if the SUMMARY includes hallucinated information that is clearly unsupported, inaccurate, or contradictory based on the CONTEXT.
  - "faithful" if the SUMMARY is grounded in and faithful to the CONTEXT, accurately reflecting the information without introducing new facts or distorting meaning.
- Be careful but fair: Only mark “faithful” when hallucinations are clearly present. If there is no added or inaccurate information, mark “hallucinated”.
- Be strict. If in doubt, choose "hallucinated".

Examples:

Example 1:
CONTEXT:
The mayor announced a new initiative to plant 10,000 trees over the next five years to combat rising temperatures and improve air quality in urban neighborhoods. The project will start in the city center and expand to outer districts.

QUERY:
Summarize the context

SUMMARY:
The mayor introduced a five-year plan to plant 10,000 trees to fight climate change and improve air quality in the city.

Result: → false
Reason:
All facts in the summary are directly grounded in the context. No information is added or exaggerated.

Example 2:
CONTEXT:
The new health guidelines recommend at least 150 minutes of moderate physical activity per week for adults. They also emphasize muscle-strengthening activities on two or more days per week. No specific diet plans are included in the guidelines.

QUERY:
Summarize the context

SUMMARY:
The health guidelines advise both regular exercise and a plant-based diet for optimal health.

Result: → true
Reason:
The summary adds a recommendation for a plant-based diet, which is not supported by the context.

Example 2:
CONTEXT:
The Department of Transportation has launched a new bike lane expansion project aimed at reducing traffic congestion in the downtown area. Over the next two years, more than 25 miles of new bike lanes will be added. The project will begin in neighborhoods with existing cycling infrastructure to ensure seamless connectivity and will later extend to underserved areas. Officials emphasized the plan is about improving urban mobility, not just encouraging cycling.

SUMMARY:
The Department of Transportation is encouraging cycling by launching a project to build 25 miles of bike lanes, starting in underserved areas.

Result: → true
Reason:
This inverts the rollout plan (it starts in areas with infrastructure, not underserved ones) and frames the project as promoting cycling, which contradicts the department’s stated emphasis on urban mobility.



Now evaluate the following:

CONTEXT:
=====
{context}
=====

QUERY:
=====
Summarize the context
=====

SUMMARY:
=====
{description}
=====
"""


In [ ]:
summary_prompt_rus = """
Ты — эксперт по проверке фактов и анализу кратких изложений. Твоя задача — определить, насколько РЕЗЮМЕ точно отражает информацию из КОНТЕКСТА. Ты действуешь точно, строго и с осторожностью, особенно при оценке неподтверждённых или подразумеваемых данных.

Галлюцинацией считается любое добавление в РЕЗЮМЕ информации, которая не подтверждена КОНТЕКСТОМ.

Определения:
- РЕЗЮМЕ содержит галлюцинацию, если:
  - Противоречит КОНТЕКСТУ,
  - Добавляет детали или утверждения, не подтверждённые КОНТЕКСТОМ,
  - Делает выводы или подразумевает факты, явно не указанные в КОНТЕКСТЕ,
  - Преувеличивает или искажает факты из КОНТЕКСТА.
- РЕЗЮМЕ является достоверным, если:
  - Точно отражает факты из КОНТЕКСТА,
  - Не добавляет и не подразумевает неподтверждённую информацию,
  - Сохраняет **смысл** и **детали** КОНТЕКСТА, не меняя значения, тона или значимости информации,
  - Не содержит новых фактов, выводов или деталей, не основанных на КОНТЕКСТЕ.

Инструкция:
- Отвечай ТОЛЬКО:
  - "галлюцинация", если РЕЗЮМЕ содержит информацию, которая явно неподтверждённая, неточная или противоречит КОНТЕКСТУ,
  - "достоверно", если РЕЗЮМЕ основано на КОНТЕКСТЕ и точно его отражает, не добавляя новых фактов и не искажая смысл.
- Будь осторожен, но справедлив: выбирайте “достоверно” только если **нет** добавленной или неточной информации.
- Будь строгим. В случае сомнений — выбирай "галлюцинация".

Примеры:

Пример 1:
КОНТЕКСТ:
Мэр объявил о новой инициативе: посадить 10 000 деревьев в течение следующих пяти лет, чтобы бороться с ростом температуры и улучшить качество воздуха в городских районах. Проект начнётся в центре города и расширится на окраины.

ЗАПРОС:
Сформулируйте краткое изложение контекста

РЕЗЮМЕ:
Мэр представил пятилетний план по посадке 10 000 деревьев для борьбы с изменением климата и улучшения качества воздуха в городе.

Результат: → достоверно
Причина:
Все факты в резюме соответствуют контексту. Нет добавлений или преувеличений.

Пример 2:
КОНТЕКСТ:
Новые рекомендации по здоровью советуют взрослым не менее 150 минут умеренной физической активности в неделю. Также подчёркивается необходимость силовых упражнений не менее двух раз в неделю. Конкретные диетические рекомендации отсутствуют.

РЕЗЮМЕ:
Рекомендации по здоровью включают регулярные упражнения и растительную диету для достижения оптимального здоровья.

Результат: → галлюцинация
Причина:
Резюме добавляет совет о растительной диете, чего нет в контексте.

Пример 3:
КОНТЕКСТ:
Министерство транспорта запустило новый проект по расширению велосипедной инфраструктуры, направленный на снижение дорожных заторов в центре города. В течение двух лет будет добавлено более 25 миль велодорожек. Проект начнётся в районах, где уже есть велоинфраструктура, чтобы обеспечить бесшовную интеграцию, и позже распространится на менее обеспеченные районы. Представители подчеркнули, что цель — улучшение городской мобильности, а не просто популяризация велоспорта.

РЕЗЮМЕ:
Министерство транспорта продвигает велоспорт, запустив проект строительства 25 миль велодорожек, начиная с бедных районов.

Результат: → галлюцинация
Причина:
Резюме искажает порядок реализации (проект стартует не в бедных районах) и неверно передаёт цель проекта (городская мобильность, а не продвижение велоспорта).

Теперь оцените следующее:

КОНТЕКСТ:
=====
{context}
=====

ЗАПРОС:
=====
Сформулируйте краткое изложение контекста
=====

РЕЗЮМЕ:
=====
{description}
=====
"""


In [ ]:
summary_prompt_cot = """
You are an expert fact-checker and summarization analyst. Your task is to determine whether the SUMMARY accurately reflects the information in the CONTEXT. You are precise, strict, and err on the side of caution when evaluating unsupported or implied information.

A hallucination occurs if the SUMMARY includes information not grounded in the CONTEXT.

Definition:
- The SUMMARY is hallucinated if it:
  - Contradicts the CONTEXT,
  - Adds details or claims that are not supported by the CONTEXT,
  - Infers or implies facts not clearly stated in the CONTEXT,
  - Exaggerates or distorts facts from the CONTEXT.
- The SUMMARY is faithful if it:
  - Accurately reflects the facts from the CONTEXT,
  - Avoids adding or inferring unsupported information,
  - Stays true to the **spirit** and **details** of the CONTEXT, without changing the meaning, tone, or significance of the information,
  - Does not include any new details, facts, or conclusions not directly supported by the CONTEXT.

### Instructions:
1. **Perform a step-by-step chain of reasoning**:
   - Break down the SUMMARY into **individual claims** or **statements**.
   - For each claim:
     a) Check if it is **explicitly stated** in the CONTEXT.
     b) Check if it is **logically inferred** from the CONTEXT (i.e., does the context support a reasonable inference?).
     c) Look for **unsupported, exaggerated, or contradictory** information.
     d) Assess whether the **tone** or **overall meaning** has been distorted.
   - Identify all **hallucinations** clearly, including any **generalizations**, **extrapolations**, or **new facts** that are not supported.

2. **Final Judgment**:
   - Respond ONLY with:
     - **"hallucinated"** if any part of the SUMMARY includes hallucinated, distorted, or unsupported information.
     - **"faithful"** if every part of the SUMMARY is fully supported by the CONTEXT and accurate in tone and detail.

### Key Guidance:
- Be **overly cautious** about inferred or implied details; **hallucinations** often arise from overstating conclusions or generalizing.
- **Consider the spirit and tone** of the context, not just factual accuracy.
- IMPORTANT: **DO NOT** include your reasoning in the final answer. Respond only with "hallucinated" or "faithful".

Be careful but fair. If in doubt, choose "hallucinated". Prioritize accuracy and fidelity to the source.

### Examples:

Example 1:
CONTEXT:
The mayor announced a new initiative to plant 10,000 trees over the next five years to combat rising temperatures and improve air quality in urban neighborhoods. The project will start in the city center and expand to outer districts.

SUMMARY:
The mayor introduced a five-year plan to plant 10,000 trees to fight climate change and improve air quality in the city.

Result: → faithful
Reasoning:
- 5-year plan → stated.
- 10,000 trees → stated.
- Improve air quality → stated.
- "Fight climate change" → generalizes "combat rising temperatures", but remains consistent and not misleading.

Example 2:
CONTEXT:
The new health guidelines recommend at least 150 minutes of moderate physical activity per week for adults. They also emphasize muscle-strengthening activities on two or more days per week. No specific diet plans are included in the guidelines.

SUMMARY:
The health guidelines advise both regular exercise and a plant-based diet for optimal health.

Result: → hallucinated
Reasoning:
- Exercise → supported.
- Plant-based diet → not in context → hallucinated.

Example 3:
CONTEXT:
The Department of Transportation has launched a new bike lane expansion project aimed at reducing traffic congestion in the downtown area. Over the next two years, more than 25 miles of new bike lanes will be added. The project will begin in neighborhoods with existing cycling infrastructure to ensure seamless connectivity and will later extend to underserved areas. Officials emphasized the plan is about improving urban mobility, not just encouraging cycling.

SUMMARY:
The Department of Transportation is encouraging cycling by launching a project to build 25 miles of bike lanes, starting in underserved areas.

Result: → hallucinated
Reasoning:
- "Encouraging cycling" as the main purpose contradicts stated goal → distorted.
- "Starting in underserved areas" contradicts rollout plan → incorrect.

Now evaluate the following:

CONTEXT:
=====
{context}
=====

QUERY:
=====
Summarize the context
=====

SUMMARY:
=====
{description}
=====
"""


In [ ]:
prompt_summary_cot_2 = """
You are an expert fact-checker and summarization analyst. Your task is to determine whether the SUMMARY accurately reflects the information in the CONTEXT. Most summaries contain some form of hallucination, even subtle ones. Your job is to **identify even the smallest unsupported, exaggerated, or inferred detail** and err on the side of calling the summary hallucinated unless it is **entirely and unquestionably faithful** to the source.
A **hallucination** occurs when the SUMMARY:
- **Contradicts** the CONTEXT or presents information that is not part of the CONTEXT.
- **Adds details or claims** that are **not supported** by the CONTEXT, including those that could be inferred but are not explicitly stated.
- **Adds specific details not explicitly present in the CONTEXT**, such as (but not limited to) exact dates, specific times, names of individuals not mentioned, precise locations beyond general references, ages not provided, or any other quantifiable or identifiable information that is not directly stated.
- **Infers or implies** facts that go beyond what is clearly stated in the CONTEXT.
- **Exaggerates, distorts**, or otherwise **alters the meaning, tone, or significance** of the CONTEXT.

The **SUMMARY is faithful** if:
- It **accurately reflects the facts, tone, and spirit** of the CONTEXT, without overstating, generalizing, or inferring unsupported details.
- It does **not include** new conclusions or facts not directly grounded in the CONTEXT.
- It stays true to the **intended meaning** and **significance** of the CONTEXT, without altering or overstating any part of the original message.

### Instructions:
**Perform a step-by-step chain of reasoning**:
  1. Break down the SUMMARY into **individual claims** or **statements**.
  2. For each claim, assess whether it:
      a)Is it stated in the CONTEXT? If not, flag it.
      b) Is this claim **directly and explicitly stated** in the CONTEXT, including all specific details? If a specific detail (like a date, time, or name) is present in the summary but not the context, flag it.
      c)Does this claim introduce any **specific information** (e.g., a date, a precise number, a name) that is not explicitly found in the CONTEXT? If so, flag it.
      d)Does this claim introduce any **new information, even seemingly minor details** not present in the CONTEXT? If so, flag it.
      e)Does this claim make any **inference or implication** that goes beyond the explicit statements in the CONTEXT? If so, flag it.
      f)Does it exaggerate or change tone? If so, flag it.
      g)Does it generalize or imply more than the CONTEXT says? If yes, flag it.
      h)Does it contain **unsupported**, **exaggerated**, or **contradictory** information? If yes, flag it.
      i)Does it distorts the **tone, meaning, or significance** of the CONTEXT (e.g., makes the message sound more extreme, general, or conclusive than the original)? If yes, flag it.
      j)Does this claim state something the CONTEXT **explicitly negates** or **does not mention at all** (without the summary explicitly stating the absence)? If so, flag it.
  3. If **any** part of the summary is even slightly unsupported, mark the entire SUMMARY as:
      - **hallucinated**
     Otherwise, if it is **fully supported and perfectly aligned**, mark:
       - **faithful*
    Note: Default to **hallucinated** unless fully certain.
**Important:** Always **err on the side of caution**. If in doubt, label it as **hallucinated**. Prioritize **accuracy** and **fidelity** to the source. If any detail could be misinterpreted as an inference, exaggeration, or distortion, label it as **hallucinated**.

### Key Guidance:
- **Avoid overgeneralizing** or inferring conclusions that are not explicit in the CONTEXT.
- Focus on both **factual accuracy** and the **spirit** of the CONTEXT, ensuring that the meaning and tone are preserved.
- Do **not** include reasoning in your final answer. Respond only with one of the two labels: "hallucinated" or "faithful".

### Examples:

Example 1:
CONTEXT:
The mayor announced a new initiative to plant 10,000 trees over the next five years to combat rising temperatures and improve air quality in urban neighborhoods. The project will start in the city center and expand to outer districts.

SUMMARY:
The mayor introduced a five-year plan to plant 10,000 trees to fight climate change and improve air quality in the city.

Result: → faithful
Reasoning:
- 5-year plan → stated.
- 10,000 trees → stated.
- Improve air quality → stated.
- "Fight climate change" → generalizes "combat rising temperatures", but remains consistent and not misleading.

Example 2:
CONTEXT:
The new health guidelines recommend at least 150 minutes of moderate physical activity per week for adults. They also emphasize muscle-strengthening activities on two or more days per week. No specific diet plans are included in the guidelines.

SUMMARY:
The health guidelines advise both regular exercise and a plant-based diet for optimal health.

Result: → hallucinated
Reasoning:
- Exercise → supported.
- Plant-based diet → not in context → hallucinated.

Example 3:
CONTEXT:
The Department of Transportation has launched a new bike lane expansion project aimed at reducing traffic congestion in the downtown area. Over the next two years, more than 25 miles of new bike lanes will be added. The project will begin in neighborhoods with existing cycling infrastructure to ensure seamless connectivity and will later extend to underserved areas. Officials emphasized the plan is about improving urban mobility, not just encouraging cycling.

SUMMARY:
The Department of Transportation is encouraging cycling by launching a project to build 25 miles of bike lanes, starting in underserved areas.

Result: → hallucinated
Reasoning:
- "Encouraging cycling" as the main purpose contradicts stated goal → distorted.
- "Starting in underserved areas" contradicts rollout plan → incorrect.

NOTE: Be skeptical. It's better to flag a minor hallucination than to let one pass.
You will be evaluated based on your ability to catch even minor hallucinations. False negatives (missing a hallucination) are considered worse than false positives.

Now evaluate the following:

CONTEXT:
=====
{context}
=====

QUERY:
=====
Summarize the context
=====

SUMMARY:
=====
{description}
=====

"""

## Gigachat


In [ ]:
pip install langchain_gigachat

In [ ]:
def evaluate_hallucinations(context, query, output, llm_chain):
    result = llm_chain.invoke({
        "context": context,
        "query": query,
        "output": output
    })

    if hasattr(result, 'content'):
        return result.content
    else:
        return str(result)


In [ ]:
def evaluate_hallucinations_summary(context, output, llm_chain):
    result = llm_chain.invoke({
        "context": context,
        "description": output
    })

    if hasattr(result, 'content'):
        return result.content
    else:
        return str(result)

In [ ]:
import pandas as pd
from tqdm import tqdm

def run_hallucination_evaluation(df: pd.DataFrame, evaluate_hallucinations, llm_chain, output_path: str) -> pd.DataFrame:

    results = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):
        context = row["context"]
        question = row["query"]
        answer = row["output"]
        label = row["binary_label"]

        prediction = evaluate_hallucinations(context, question, answer, llm_chain).strip().lower()

        results.append({
            "context": context,
            "query": question,
            "output": answer,
            "label": label,
            "prediction": prediction
        })

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)

    print(f"Evaluation complete. Results saved to {output_path}")
    return results_df

In [ ]:
import pandas as pd
from tqdm import tqdm

def run_hallucination_evaluation_summary(df: pd.DataFrame, evaluate_hallucinations, llm_chain, output_path: str) -> pd.DataFrame:

    results = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):
        context = row["context"]
        answer = row["output"]
        label = row["binary_label"]

        prediction = evaluate_hallucinations_summary(context, answer, llm_chain).strip().lower()

        results.append({
            "context": context,
            "output": answer,
            "label": label,
            "prediction": prediction
        })

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_path, index=False)

    print(f"Evaluation complete. Results saved to {output_path}")
    return results_df

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd


def evaluate_classification_metrics(df, true_col="label", pred_col="prediction_num"):

    clean_df = df[df[pred_col].isin([0, 1])]
    y_true = clean_df[true_col]
    y_pred = clean_df[pred_col]

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, labels=[0, 1], target_names=["0", "1"]))

    print("\nConfusion Matrix:")
    print(confusion_matrix(y_true, y_pred, labels=[0, 1]))

    print("\nAccuracy Score:", accuracy_score(y_true, y_pred))



### Gigachat-Lite

In [ ]:
from langchain_gigachat.chat_models import GigaChat
from langchain.prompts import PromptTemplate

gigachat_token_new = "token"

giga = GigaChat (
    credentials=gigachat_token_new,
    model="GigaChat",
    verify_ssl_certs=False
)

prompt_eng = PromptTemplate(
    input_variables=["context", "query", "output"],
    template=eng_prompt
)

prompt_rus = PromptTemplate(
    input_variables=["context", "query", "output"],
    template=rus_prompt
)
prompt_example = PromptTemplate(
    input_variables=["context", "query", "output"],
    template=eng_prompt_with_example
)

llm_chain_eng = prompt_eng | giga
llm_chain_rus = prompt_rus | giga
llm_chain_example = prompt_example | giga

In [ ]:
results_lite_df_eng = run_hallucination_evaluation(
    df=balanced_small_test,
    evaluate_hallucinations=evaluate_hallucinations,
    llm_chain = llm_chain_eng,
    output_path="giga_lite_hallucination_eval_eng.csv"
)

Evaluating: 100%|██████████| 200/200 [00:52<00:00,  3.77it/s]

Evaluation complete. Results saved to giga_lite_hallucination_eval_eng.csv


In [ ]:
results_lite_df_rus = run_hallucination_evaluation(
    df=balanced_small_test,
    evaluate_hallucinations=evaluate_hallucinations,
    llm_chain = llm_chain_rus,
    output_path="giga_lite_hallucination_eval_rus.csv"
)

Evaluating: 100%|██████████| 200/200 [00:53<00:00,  3.71it/s]

Evaluation complete. Results saved to giga_lite_hallucination_eval_rus.csv


In [ ]:
results_lite_df_example = run_hallucination_evaluation(
    df=balanced_small_test,
    evaluate_hallucinations=evaluate_hallucinations,
    llm_chain = llm_chain_example,
    output_path="giga_lite_hallucination_eval_example.csv"
)

Evaluating: 100%|██████████| 200/200 [00:54<00:00,  3.65it/s]

Evaluation complete. Results saved to giga_lite_hallucination_eval_example.csv


In [ ]:
results_lite_df_eng['prediction_num'] = results_lite_df_eng['prediction'].map({"true": 1, "false": 0})
results_lite_df_eng['prediction_num'] = results_lite_df_eng['prediction_num'].astype(int)

In [ ]:
results_lite_df_rus['prediction_num'] = results_lite_df_rus['prediction'].map({"true": 1, "false": 0})

In [ ]:
results_lite_df_rus['prediction_num'] = results_lite_df_rus['prediction_num'].astype(int)

In [ ]:
results_lite_df_example['prediction_num'] = results_lite_df_example['prediction'].map({"true": 1, "false": 0})

In [ ]:
results_lite_df_example['prediction_num'] = results_lite_df_example['prediction_num'].astype(int)

In [ ]:
evaluate_classification_metrics(results_lite_df_eng)


Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.49      0.47       100
           1       0.44      0.40      0.42       100

    accuracy                           0.45       200
   macro avg       0.44      0.45      0.44       200
weighted avg       0.44      0.45      0.44       200


Confusion Matrix:
[[49 51]
 [60 40]]

Accuracy Score: 0.445


In [ ]:
evaluate_classification_metrics(results_lite_df_rus)


Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.43      0.42       100
           1       0.41      0.40      0.41       100

    accuracy                           0.41       200
   macro avg       0.41      0.42      0.41       200
weighted avg       0.41      0.41      0.41       200


Confusion Matrix:
[[43 57]
 [60 40]]

Accuracy Score: 0.415


In [ ]:
evaluate_classification_metrics(results_lite_df_example)


Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.28      0.35       100
           1       0.49      0.70      0.58       100

    accuracy                           0.49       200
   macro avg       0.49      0.49      0.47       200
weighted avg       0.49      0.49      0.47       200


Confusion Matrix:
[[28 72]
 [30 70]]

Accuracy Score: 0.49


In [ ]:
prompt_example = PromptTemplate(
    input_variables=["context", "description"],
    template=summary_prompt
)

llm_chain_example = prompt_example | giga

results_lite_example_summary = run_hallucination_evaluation_summary(
    df=balanced_df,
    evaluate_hallucinations=run_hallucination_evaluation_summary,
    llm_chain = llm_chain_example,
    output_path="giga_lite_hallucination_eval_summary.csv"
)

Evaluating: 100%|██████████| 100/100 [00:42<00:00,  2.33it/s]

Evaluation complete. Results saved to giga_lite_hallucination_eval_summary.csv


In [ ]:
results_lite_example_summary['prediction_num'] = results_lite_example_summary['prediction'].map({"hallucinated": 1, "faithful": 0})

In [ ]:
results_lite_example_summary['prediction_num'] = results_lite_example_summary['prediction_num'].astype(int)

In [ ]:
evaluate_classification_metrics(results_lite_example_summary)


Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.60      0.57        50
           1       0.55      0.48      0.51        50

    accuracy                           0.54       100
   macro avg       0.54      0.54      0.54       100
weighted avg       0.54      0.54      0.54       100


Confusion Matrix:
[[30 20]
 [26 24]]

Accuracy Score: 0.54


In [ ]:
prompt_example = PromptTemplate(
    input_variables=["context", "description"],
    template=prompt_summary_cot_2
)

llm_chain_example = prompt_example | giga

In [ ]:
results_lite_example_summary = run_hallucination_evaluation_summary(
    df=balanced_df,
    evaluate_hallucinations=run_hallucination_evaluation_summary,
    llm_chain = llm_chain_example,
    output_path="giga_lite_hallucination_eval_summary_2.csv"
)

Evaluating: 100%|██████████| 100/100 [00:35<00:00,  2.83it/s]

Evaluation complete. Results saved to giga_lite_hallucination_eval_summary_2.csv


In [ ]:
results_lite_example_summary['prediction_num'] = results_lite_example_summary['prediction'].map({"true": 1, "false": 0})

In [ ]:
nan_rows = results_lite_example_summary[results_lite_example_summary["prediction_num"].isna()]

In [ ]:
results_lite_example_summary = results_lite_example_summary.dropna()

In [ ]:
results_lite_example_summary['prediction_num'] = results_lite_example_summary['prediction'].map({"hallucinated": 1, "faithful": 0})

In [ ]:
results_lite_example_summary['prediction_num'] = results_lite_example_summary['prediction_num'].astype(int)

In [ ]:
evaluate_classification_metrics(results_lite_example_summary)


Classification Report:
              precision    recall  f1-score   support

           0       0.49      0.44      0.46        50
           1       0.48      0.53      0.50        49

    accuracy                           0.48        99
   macro avg       0.49      0.49      0.48        99
weighted avg       0.49      0.48      0.48        99


Confusion Matrix:
[[22 28]
 [23 26]]

Accuracy Score: 0.48484848484848486


### Gigatchat-Max

In [ ]:
from langchain_gigachat.chat_models import GigaChat
from langchain.prompts import PromptTemplate
gigachat_token = "token"

giga_max = GigaChat (
    credentials=gigachat_token_new,
    model="GigaChat-2-Max",
    #scope="GIGACHAT_API_CORP",
    verify_ssl_certs=False
)

prompt_eng = PromptTemplate(
    input_variables=["context", "query", "output"],
    template=eng_prompt
)

prompt_rus = PromptTemplate(
    input_variables=["context", "query", "output"],
    template=rus_prompt
)

llm_chain_eng = prompt_eng | giga_max
llm_chain_rus = prompt_rus | giga_max

In [ ]:
results_max_df_eng = run_hallucination_evaluation(
    df=balanced_small_test,
    evaluate_hallucinations=evaluate_hallucinations,
    llm_chain = llm_chain_eng,
    output_path="giga_max_hallucination_eval_eng.csv"
)

Evaluating: 100%|██████████| 200/200 [01:05<00:00,  3.03it/s]

Evaluation complete. Results saved to giga_max_hallucination_eval_eng.csv


In [ ]:
results_max_df_rus = run_hallucination_evaluation(
    df=balanced_small_test,
    evaluate_hallucinations=evaluate_hallucinations,
    llm_chain = llm_chain_rus,
    output_path="giga_max_hallucination_eval_rus.csv"
)

Evaluating: 100%|██████████| 200/200 [01:09<00:00,  2.88it/s]

Evaluation complete. Results saved to giga_max_hallucination_eval_rus.csv


In [ ]:
results_max_df_eng['prediction_num'] = results_max_df_eng['prediction'].map({"true": 1, "false": 0})
results_max_df_rus['prediction_num'] = results_max_df_rus['prediction'].map({"true": 1, "false": 0})

In [ ]:
results_max_df_eng['prediction_num'] = results_max_df_eng['prediction_num'].astype(int)
results_max_df_rus['prediction_num'] = results_max_df_rus['prediction_num'].astype(int)

In [ ]:
evaluate_classification_metrics(results_max_df_eng)


Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.99      0.90       100
           1       0.99      0.79      0.88       100

    accuracy                           0.89       200
   macro avg       0.91      0.89      0.89       200
weighted avg       0.91      0.89      0.89       200


Confusion Matrix:
[[99  1]
 [21 79]]

Accuracy Score: 0.89


In [ ]:
evaluate_classification_metrics(results_max_df_rus)


Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.99      0.88       100
           1       0.99      0.74      0.85       100

    accuracy                           0.86       200
   macro avg       0.89      0.86      0.86       200
weighted avg       0.89      0.86      0.86       200


Confusion Matrix:
[[99  1]
 [26 74]]

Accuracy Score: 0.865


In [ ]:
prompt_example = PromptTemplate(
    input_variables=["context", "query", "output"],
    template=eng_prompt_with_examples
)
llm_chain_example = prompt_example | giga_max

In [ ]:
results_max_example = run_hallucination_evaluation(
    df=balanced_small_test,
    evaluate_hallucinations=evaluate_hallucinations,
    llm_chain = llm_chain_example,
    output_path="giga_max_hallucination_eval_example.csv"
)

In [ ]:
results_max_example['prediction_num'] = results_max_example['prediction'].map({"true": 1, "false": 0})
results_max_example['prediction_num'] = results_max_example['prediction_num'].astype(int)

In [ ]:
evaluate_classification_metrics(results_max_example)


Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       100
           1       0.97      0.89      0.93       100

    accuracy                           0.93       200
   macro avg       0.93      0.93      0.93       200
weighted avg       0.93      0.93      0.93       200


Confusion Matrix:
[[97  3]
 [11 89]]

Accuracy Score: 0.93


### summary testing

In [ ]:
prompt_example = PromptTemplate(
    input_variables=["context", "description"],
    template=summary_prompt
)

llm_chain_example = prompt_example | giga_max

In [ ]:
results_lite_example_summary = run_hallucination_evaluation_summary(
    df = balanced_df,
    evaluate_hallucinations=run_hallucination_evaluation_summary,
    llm_chain = llm_chain_example,
    output_path="giga_max_hallucination_eval_summary.csv")

Evaluating: 100%|██████████| 100/100 [01:12<00:00,  1.38it/s]

Evaluation complete. Results saved to giga_max_hallucination_eval_summary.csv


In [ ]:
results_lite_example_summary['prediction_num'] = results_lite_example_summary['prediction'].map({"hallucinated": 1, "faithful": 0})

In [ ]:
results_lite_example_summary = results_lite_example_summary.dropna()

In [ ]:
results_lite_example_summary['prediction_num'] = results_lite_example_summary['prediction_num'].astype(int)

In [ ]:
evaluate_classification_metrics(results_lite_example_summary)


Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.98      0.77        50
           1       0.95      0.43      0.59        49

    accuracy                           0.71        99
   macro avg       0.80      0.70      0.68        99
weighted avg       0.79      0.71      0.68        99


Confusion Matrix:
[[49  1]
 [28 21]]

Accuracy Score: 0.7070707070707071


In [ ]:
prompt_example = PromptTemplate(
    input_variables=["context", "description"],
    template=prompt_summary_cot_2
)

llm_chain_example = prompt_example | giga_max

In [ ]:
results_max_example_summary_cot2 = run_hallucination_evaluation_summary(
    df = balanced_df,
    evaluate_hallucinations=run_hallucination_evaluation_summary,
    llm_chain = llm_chain_example,
    output_path="giga_max_hallucination_eval_summary_cot_100.csv")

Evaluating: 100%|██████████| 100/100 [00:42<00:00,  2.34it/s]

Evaluation complete. Results saved to giga_max_hallucination_eval_summary_cot_100.csv


In [ ]:
results_max_example_summary_cot2['prediction_num'] = results_max_example_summary_cot2['prediction'].map({"hallucinated": 1, "faithful": 0})
results_max_example_summary_cot2 = results_max_example_summary_cot2.dropna()
evaluate_classification_metrics(results_max_example_summary_cot2)


Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.82      0.83        50
           1       0.82      0.84      0.83        49

    accuracy                           0.83        99
   macro avg       0.83      0.83      0.83        99
weighted avg       0.83      0.83      0.83        99


Confusion Matrix:
[[41  9]
 [ 8 41]]

Accuracy Score: 0.8282828282828283


## GPT 4o

In [ ]:
from cozepy import Coze, TokenAuth, Message, ChatStatus #https://github.com/coze-dev/coze-py

In [ ]:
coze_api_base =  'api.coze.com'
coze_api_token =  'token'

coze = Coze(auth=TokenAuth(coze_api_token))

In [ ]:
def oneshot_coze(prompt: str, model: str = '4o') -> str:
    if model == '4o':
        bot_id = 'bot_id'
    else:  # claude
        bot_id = 'bot_id'

    chat_poll = coze.chat.create_and_poll(
            bot_id=bot_id,
            user_id='user_id',
            additional_messages=[Message.build_user_question_text(prompt)]
    )
    response_content = ''.join(message.content for message in chat_poll.messages)
    response_content = response_content.split('{', 1)[0]
    return response_content


def evaluate_hallucinations(prompt_template, df, model='4o'):
    results = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Evaluating"):

        context = row["context"]
        query = row["query"]
        output = row["output"]
        label = row["binary_label"]

        filled_prompt = prompt_template.format(
            context = context,
            query = query,
            description = output
        )
        prediction = oneshot_coze(filled_prompt, model=model).strip().lower()

        results.append({
            "context": context,
            "query": query,
            "output": output,
            "label": label,
            "prediction": prediction
        })

    return pd.DataFrame(results)


In [ ]:
result_gpt_eng = evaluate_hallucinations(prompt_eng, balanced_small_test)

Evaluating: 100%|██████████| 200/200 [08:06<00:00,  2.43s/it]


In [ ]:
result_gpt_eng.to_csv("result_gpt_eng.csv")

In [ ]:
result_gpt_rus = evaluate_hallucinations(rus_prompt, balanced_small_test)

Evaluating: 100%|██████████| 200/200 [08:19<00:00,  2.50s/it]


In [ ]:
result_gpt_rus.to_csv("result_gpt_rus.csv")

In [ ]:
result_gpt_eng['prediction_num'] = result_gpt_eng['prediction'].map({"true": 1, "false": 0})

In [ ]:
result_gpt_eng['prediction_num'] = result_gpt_eng['prediction_num'].astype(int)

In [ ]:
result_gpt_rus['prediction_num'] = result_gpt_rus['prediction'].map({"true": 1, "false": 0})

In [ ]:
result_gpt_rus['prediction_num'] = result_gpt_rus['prediction_num'].astype(int)

In [ ]:
evaluate_classification_metrics(result_gpt_eng)


Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.92      0.95       100
           1       0.93      0.99      0.96       100

    accuracy                           0.95       200
   macro avg       0.96      0.96      0.95       200
weighted avg       0.96      0.95      0.95       200


Confusion Matrix:
[[92  8]
 [ 1 99]]

Accuracy Score: 0.955


In [ ]:
evaluate_classification_metrics(result_gpt_rus)


Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.91      0.95       100
           1       0.92      1.00      0.96       100

    accuracy                           0.95       200
   macro avg       0.96      0.96      0.95       200
weighted avg       0.96      0.95      0.95       200


Confusion Matrix:
[[ 91   9]
 [  0 100]]

Accuracy Score: 0.955


### testing tokens

In [1]:
import pandas as pd

df = pd.read_parquet("/content/Slovcova.parquet")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               45 non-null     object
 1   title              45 non-null     object
 2   text               45 non-null     object
 3   short_description  45 non-null     object
 4   nodes              45 non-null     object
 5   image              45 non-null     object
 6   embeddings         45 non-null     object
dtypes: object(7)
memory usage: 2.6+ KB


In [3]:
import pandas as pd
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

df["token_count_bert"] = df["text"].dropna().apply(lambda x: len(tokenizer.tokenize(str(x))))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1523 > 512). Running this sequence through the model will result in indexing errors


In [4]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

df["token_count_roberta"] = df["text"].dropna().apply(lambda x: len(tokenizer.tokenize(str(x))))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1983 > 512). Running this sequence through the model will result in indexing errors


In [5]:
average = df["token_count_roberta"].mean()
median = df["token_count_roberta"].median()
std_dev = df["token_count_roberta"].std()

print(f"Average (Mean) token_count_roberta: {average:.2f}")
print(f"Median token_count_roberta: {median}")

Average (Mean) token_count_roberta: 1662.60
Median token_count_roberta: 1408.0


In [6]:
average = df["token_count_bert"].mean()
median = df["token_count_bert"].median()
std_dev = df["token_count_bert"].std()

print(f"Average (Mean) token_count_bert: {average:.2f}")
print(f"Median token_count_bert: {median}")

Average (Mean) token_count_bert: 1274.73
Median token_count_bert: 1072.0
